In [1]:
!pip install langchain-community
!pip install langchain
!pip install sentence-transformers
!pip install pdfplumber
!pip install chromadb
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.7/367.7 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.3.23
    Uninstalling langsmith-0.3.23:
      Successfully uninstalled langsmith-0.3.23
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.50
    Uninstalling langchain-core-0.3.50:
      Successfully uninstalled langchain-core-0.3.50
  A

In [2]:
import torch
from transformers import AutoTokenizer, pipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders.pdf import PDFPlumberLoader
from langchain.text_splitter import CharacterTextSplitter, TokenTextSplitter
from langchain.vectorstores import Chroma
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

2025-06-30 15:54:24.139921: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751298864.543388      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751298864.652333      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
def create_sbert_mpnet():
  model = "sentence-transformers/all-mpnet-base-v2"
  device = "cuda" if torch.cuda.is_available() else "cpu"
  return HuggingFaceEmbeddings(model_name = model, model_kwargs = {"device": device})

In [4]:
def create_flan_t5_base(load_in_8bit):
  model = "google/flan-t5-xl"
  tokenizer = AutoTokenizer.from_pretrained(model)
  return pipeline(
      task = "text2text-generation",
      model = model,
      tokenizer = tokenizer,
      max_new_tokens = 100,
      device_map = "auto",
      model_kwargs = {"load_in_8bit": load_in_8bit,
                      "temperature": 0.95}
  )

In [5]:
embedding = create_sbert_mpnet()
llm = create_flan_t5_base(load_in_8bit = False)

/tmp/ipykernel_35/1598665095.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  return HuggingFaceEmbeddings(model_name = model, model_kwargs = {"device": device})


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cuda:0


In [51]:
_question = "What is Rubika application? let't think step by step to get result."
prompt = "Answer the following question. \n" + _question
res = llm(prompt)
print(res)

[{'generated_text': 'Rubika is a free and open-source software application for generating rubrics from a set of digits. Rubrics are used to represent the digits of a number. Rubrics are used to represent the digits of a number. So, the final answer is Rubrics.'}]


In [52]:
_question = "What is Rubino? let's think step by step to get result."
prompt = "Answer the following question. \n" + _question
res = llm(prompt)
print(res)

[{'generated_text': 'Rubino is a surname. Rubino is a surname. So, the final answer is Rubino.'}]


In [53]:
pdf_path = "/kaggle/input/rubikadata/Rubika.pdf"
loader = PDFPlumberLoader(pdf_path)
documents = loader.load()
text_splitter = TokenTextSplitter(chunk_size = 200, chunk_overlap = 10)
texts = text_splitter.split_documents(documents)
persist_directory = "./chromadb"
vectordb = Chroma.from_documents(documents = texts, embedding = embedding, persist_directory = persist_directory)

In [54]:
len(texts)

9

In [55]:
for t in texts[:2]:
  print(t)
  print("#" * 80)

page_content='Rubika is an Iranian messenger developed by Toska, a subsidiary of the Executive
Headquarters of the Imam's Decree .
The Android version of the platform was removed in August 1400 due to misuse of
user information and creation of fake accounts on several services, including the Play
Store . Google Play Defense Shield also identified Rubika as spyware in November
1402.
On October 16, 2017, the Iranian Broadcasting Corporation (IBRC) awarded all of its VAS and
payment industry advertising to a company called Toska, through a tender (which,
according to some advertising activists, was not disclosed), for a sum of several
hundred billion IDR. In fact, this company became the IRB’s advertising contractor in
this area through the same tender. After the contract between Toska and the IRB, the
company introduced the Rubika program under the name of Medan-e-Awal, or in fact,
a V' metadata={'source': '/kaggle/input/rubikadata/Rubika.pdf', 'file_path': '/kaggle/input/rubikadata/Rubi

In [56]:
vdb = vectordb.get(include = ['embeddings', 'documents', 'metadatas'])
num = 2
print(vdb['documents'][:num])
print(vdb['embeddings'][:num])
print(vdb['metadatas'][:num])

['\n', '\n']
[[-0.01250338  0.06143879 -0.00673447 ... -0.00193858 -0.05036446
  -0.01904947]
 [-0.01250338  0.06143879 -0.00673447 ... -0.00193858 -0.05036446
  -0.01904947]]
[{'Producer': 'Microsoft: Print To PDF', 'ModDate': "D:20250630065454-07'00'", 'page': 0, 'file_path': '/kaggle/input/pdfdataset/eat that frog.pdf', 'Title': 'Eat That Frog!', 'source': '/kaggle/input/pdfdataset/eat that frog.pdf', 'total_pages': 3, 'Author': 'Lenovo', 'CreationDate': "D:20250630065454-07'00'"}, {'Producer': 'Microsoft: Print To PDF', 'source': '/kaggle/input/pdfdataset/eat that frog.pdf', 'page': 1, 'ModDate': "D:20250630065454-07'00'", 'CreationDate': "D:20250630065454-07'00'", 'file_path': '/kaggle/input/pdfdataset/eat that frog.pdf', 'Author': 'Lenovo', 'total_pages': 3, 'Title': 'Eat That Frog!'}]


In [57]:
hf_llm = HuggingFacePipeline(pipeline = llm)
retriever = vectordb.as_retriever(search_kwargs = {"k": 1})
qa = RetrievalQA.from_chain_type(llm = hf_llm, chain_type = "stuff", retriever = retriever)

question_t5_template = """
context: {context}
question: {question}
answer:
"""
QUESTION_T5_PROMPT = PromptTemplate(
    template = question_t5_template, input_variables = ["context", "question"]
)
qa.combine_documents_chain.llm_chain.prompt = QUESTION_T5_PROMPT

In [58]:
_question = "What is Rubika application? let't think step by step to get result."
qa.combine_documents_chain.verbose = True
qa.return_source_documents = True
qa({"query": _question,})



> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': "What is Rubika application? let't think step by step to get result.",
 'result': "To answer the above question, the relevant sentence is: Rubika is an Iranian messenger developed by Toska, a subsidiary of the Executive Headquarters of the Imam's Decree . So, the final answer is an Iranian messenger developed by Toska , a subsidiary of the Executive Headquarters of the Imam 's Decree .",
 'source_documents': [Document(metadata={'file_path': '/kaggle/input/rubikadata/Rubika.pdf', 'page': 0, 'Creator': 'Microsoft® Word LTSC', 'total_pages': 3, 'CreationDate': "D:20250630101101-07'00'", 'Producer': 'Microsoft® Word LTSC', 'source': '/kaggle/input/rubikadata/Rubika.pdf', 'Author': 'Ali Khademi Khoubani', 'ModDate': "D:20250630101101-07'00'"}, page_content="Rubika is an Iranian messenger developed by Toska, a subsidiary of the Executive\nHeadquarters of the Imam's Decree .\nThe Android version of the platform was removed in August 1400 due to misuse of\nuser information and creati

In [59]:
_question = "What is Rubino? let's think step by step to get result."
qa.combine_documents_chain.verbose = True
qa.return_source_documents = True
qa({"query": _question,})



> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': "What is Rubino? let's think step by step to get result.",
 'result': 'To answer the above question, the relevant sentence is: Rubino Rubino is a kind of simulation of Instagram. So, the final answer is a kind of simulation of Instagram.',
 'source_documents': [Document(metadata={'Author': 'Ali Khademi Khoubani', 'file_path': '/kaggle/input/rubikadata/Rubika.pdf', 'CreationDate': "D:20250630101101-07'00'", 'page': 1, 'source': '/kaggle/input/rubikadata/Rubika.pdf', 'ModDate': "D:20250630101101-07'00'", 'total_pages': 3, 'Producer': 'Microsoft® Word LTSC', 'Creator': 'Microsoft® Word LTSC'}, page_content="Rubika messaging section for sending text, images, and voice messages for users who\nuse the internet of Irancell and Hamar-Aval operators. The pattern of calculating the\ncost of consumed internet in Rubika may change over time.\nRubino\nRubino is a kind of simulation of Instagram . Users on Rubino can share their visual\nand video content in the form of posts and stories, j

In [60]:
retriever.get_relevant_documents(_question)

[Document(metadata={'total_pages': 3, 'Producer': 'Microsoft® Word LTSC', 'source': '/kaggle/input/rubikadata/Rubika.pdf', 'Author': 'Ali Khademi Khoubani', 'file_path': '/kaggle/input/rubikadata/Rubika.pdf', 'ModDate': "D:20250630101101-07'00'", 'CreationDate': "D:20250630101101-07'00'", 'page': 1, 'Creator': 'Microsoft® Word LTSC'}, page_content="Rubika messaging section for sending text, images, and voice messages for users who\nuse the internet of Irancell and Hamar-Aval operators. The pattern of calculating the\ncost of consumed internet in Rubika may change over time.\nRubino\nRubino is a kind of simulation of Instagram . Users on Rubino can share their visual\nand video content in the form of posts and stories, just like on Instagram. Followers\ncan see the posts and stories and react to them or write a comment on the post.\nShowcase\nThe Showcase section is actually similar to the Explore section on the Instagram\nplatform, and through it, you can access the most viewed and pos

In [61]:
ret = retriever.invoke(_question)
qa.combine_documents_chain._get_inputs(ret)

{'context': "Rubika messaging section for sending text, images, and voice messages for users who\nuse the internet of Irancell and Hamar-Aval operators. The pattern of calculating the\ncost of consumed internet in Rubika may change over time.\nRubino\nRubino is a kind of simulation of Instagram . Users on Rubino can share their visual\nand video content in the form of posts and stories, just like on Instagram. Followers\ncan see the posts and stories and react to them or write a comment on the post.\nShowcase\nThe Showcase section is actually similar to the Explore section on the Instagram\nplatform, and through it, you can access the most viewed and posted content by users.\nSettings\nIn the settings section, you can manage account information such as biography and\nname and profile changes. You can also change the application's skins in the\nappearance settings section. Privacy and security settings, parental control,\nnotifications and sounds,"}

In [62]:
QUESTION_T5_PROMPT.format(question = _question, context = ret[0].page_content)

"\ncontext: Rubika messaging section for sending text, images, and voice messages for users who\nuse the internet of Irancell and Hamar-Aval operators. The pattern of calculating the\ncost of consumed internet in Rubika may change over time.\nRubino\nRubino is a kind of simulation of Instagram . Users on Rubino can share their visual\nand video content in the form of posts and stories, just like on Instagram. Followers\ncan see the posts and stories and react to them or write a comment on the post.\nShowcase\nThe Showcase section is actually similar to the Explore section on the Instagram\nplatform, and through it, you can access the most viewed and posted content by users.\nSettings\nIn the settings section, you can manage account information such as biography and\nname and profile changes. You can also change the application's skins in the\nappearance settings section. Privacy and security settings, parental control,\nnotifications and sounds,\nquestion: What is Rubino? let's think s